let's design a **Multi-Agent RAG-based Trip Planning System** that can:  
✅ Retrieve **real-time travel data** (flights, hotels, attractions)  
✅ Utilize **local knowledge bases** (pre-stored travel tips)  
✅ Leverage **online search APIs** (Google, Bing, or DuckDuckGo)  
✅ Provide structured, **well-formatted** responses  

---

### **🚀 System Architecture: Multi-Agent RAG for Trip Planning**
We'll create multiple **specialized agents**, each handling different aspects of trip planning:

1️⃣ **Flight Agent** ✈️ → Fetches flight details  
2️⃣ **Hotel Agent** 🏨 → Recommends accommodations  
3️⃣ **Attractions Agent** 🏛️ → Suggests must-visit places  
4️⃣ **Weather Agent** ⛅ → Fetches real-time weather info  
5️⃣ **Local Transport Agent** 🚆 → Provides public transport details  
6️⃣ **Dining Agent** 🍽️ → Suggests restaurants based on preferences  
7️⃣ **Budget Planner** 💰 → Estimates trip costs  
8️⃣ **Final Response Generator** 📝 → Formats a complete trip itinerary  

---

### **🛠️ Tech Stack**
- **LangChain** for RAG-based retrieval  
- **FAISS / ChromaDB** for vector storage  
- **Online Search APIs** (Google, Bing, or DuckDuckGo)  
- **OpenAI / LLaMA / Mistral** for response generation  
- **Multi-Agent Workflow** (LangGraph / AgentExecutor)  

---



### **🔨 Let's Start Building!**
I'll first create the basic **Multi-Agent RAG Framework**. Then, we'll enhance it step by step. Stay tuned! 🚀


In [35]:
import openai
import requests
from langchain.tools import Tool
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

from langchain_google_genai import ChatGoogleGenerativeAI


In [36]:
# Load API Key
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")



In [14]:
# Load the knowledge base
def load_knowledge_base():
    loader = TextLoader("data/knowledge_base.txt")
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    split_docs = text_splitter.split_documents(docs)
    vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
    return vectorstore



In [15]:
# Define agents
flight_agent = Tool(
    name="Flight Search Agent",
    func=lambda query: f"Fetching flight details for {query}...",  # Placeholder
    description="Finds flight options for the trip."
)

hotel_agent = Tool(
    name="Hotel Booking Agent",
    func=lambda query: f"Finding best hotels in {query}...",  # Placeholder
    description="Finds hotel recommendations."
)

weather_agent = Tool(
    name="Weather Agent",
    func=lambda query: f"Fetching weather details for {query}...",  # Placeholder
    description="Retrieves weather forecast."
)

attraction_agent = Tool(
    name="Attractions Finder",
    func=lambda query: f"Top places to visit in {query}: Eiffel Tower, Louvre, Notre-Dame...",  # Placeholder
    description="Suggests top tourist attractions."
)

dining_agent = Tool(
    name="Dining Agent",
    func=lambda query: f"Fetching top restaurants in {query}...",  # Placeholder
    description="Finds the best places to eat."
)

budget_agent = Tool(
    name="Budget Planner",
    func=lambda query: f"Estimating trip costs for {query}...",  # Placeholder
    description="Calculates trip expenses."
)



In [39]:
# Initialize LangChain Groq Model
#llm = ChatGroq(model_name="qwen-2.5-32b", groq_api_key=GROQ_API_KEY)
# Set up the LLM

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro-exp-03-25'", google_api_key=GEMINI_API_KEY)



In [17]:
# Load knowledge base
vectorstore = load_knowledge_base()
retrieval_agent = Tool(
    name="Knowledge Base Retriever",
    func=lambda query: vectorstore.similarity_search(query, k=3),
    description="Retrieves relevant information from the knowledge base."
)



In [21]:
# Initialize Multi-Agent System
agent_executor = initialize_agent(
    tools=[flight_agent, hotel_agent, weather_agent, attraction_agent, dining_agent, budget_agent, retrieval_agent],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    memory=ConversationBufferMemory(memory_key="chat_history")
)



In [22]:
# Query the Multi-Agent RAG System
def plan_trip(destination):
    query = f"Plan a trip to {destination}"
    response = agent_executor.run(query)
    return response



In [41]:
# Example usage
destination = "give we 5 day trip to london in june"
trip_plan = plan_trip(destination)
print(trip_plan)


For a 5-day trip to London in June, you can consider the following itinerary and recommendations:

1. **Flight and Accommodation**: I initiated a search for flight options and hotel bookings for a 5-day trip in June. You should review the options provided by the Flight Search Agent and Hotel Booking Agent for the best deals.

2. **Attractions**: Here are some top attractions in London you can visit:
   - The British Museum
   - The Tower of London
   - Buckingham Palace
   - The London Eye
   - Westminster Abbey
   - The Natural History Museum
   - Tate Modern

3. **Weather in June**: London's weather in June is generally mild, but it's advisable to check the specific forecast closer to your travel dates to pack appropriately.

4. **Itinerary**:
   - Day 1: Arrive in London and settle into your hotel. Spend the evening exploring the local area.
   - Day 2: Visit the British Museum and The Tower of London.
   - Day 3: Explore Westminster Abbey and the Houses of Parliament, then take a t


I've created the initial **Multi-Agent RAG System** for trip planning. This framework includes specialized agents for flights, hotels, attractions, dining, weather, and budgeting, along with a knowledge base retriever.

Next steps:
1️⃣ **Integrate Real-Time APIs** (Flights, Hotels, Weather, etc.)  
2️⃣ **Improve Response Formatting** (Structured itinerary output)  
3️⃣ **Enhance Retrieval with Hybrid RAG** (BM25 + FAISS)  

Let me know which feature you want to prioritize! 🚀

Got it! We'll build a **multi-agent travel itinerary planner** using:  
✅ **Multiple web search agents** (Flights, Hotels, Food, Attractions, Weather, Budget).  
✅ **Serper API** for web search.  
✅ **Gemini LLM** for final response generation.  

Each agent will **search the web for relevant information**, and at the end, the **Gemini LLM** will format and summarize everything into a well-structured itinerary.

---

### **🔹 Features of the Travel Planner**
- **Multi-agent system**: Dedicated agents for flights, hotels, food, attractions, weather, and budget.  
- **Serper API**: Fetches web results dynamically.  
- **Structured Itinerary**: LLM organizes results into a **day-by-day travel plan** if needed.  
- **LLM (Gemini)**: Generates the final user-friendly response.

---

### **🚀 Code for the Multi-Agent Travel Planner**
Here’s the full implementation:

---


In [57]:
import os
import requests
from dotenv import load_dotenv
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI


In [58]:

# Load API Keys
load_dotenv()
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")


In [59]:

# Web Search Function using Serper API
def web_search(query):
    url = "https://google.serper.dev/search"
    headers = {"X-API-KEY": SERPER_API_KEY, "Content-Type": "application/json"}
    payload = {"q": query, "num": 3}  # Fetch top 3 results
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        data = response.json()
        return "\n".join([result["link"] for result in data.get("organic", [])])
    return "No results found."


In [60]:

# Define Agents for Different Travel Tasks
flight_agent = Tool(
    name="Flight Search Agent",
    func=lambda query: web_search(f"Best flights to {query}"),
    description="Finds flight options for the trip."
)


In [61]:

hotel_agent = Tool(
    name="Hotel Booking Agent",
    func=lambda query: web_search(f"Best hotels in {query}"),
    description="Finds hotel recommendations."
)

food_agent = Tool(
    name="Food & Dining Agent",
    func=lambda query: web_search(f"Best restaurants in {query}"),
    description="Suggests top places to eat."
)

attraction_agent = Tool(
    name="Attractions Finder",
    func=lambda query: web_search(f"Top tourist attractions in {query}"),
    description="Finds must-visit attractions."
)

weather_agent = Tool(
    name="Weather Forecast Agent",
    func=lambda query: web_search(f"Weather forecast in {query} for next 5 days"),
    description="Gets the weather forecast for the trip."
)

budget_agent = Tool(
    name="Budget Planner",
    func=lambda query: web_search(f"Estimated travel costs for {query}"),
    description="Estimates the budget required for the trip."
)


In [62]:

# Initialize Gemini LLM

from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
#llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro-exp-03-25", google_api_key=GEMINI_API_KEY)
llm = ChatGroq(model_name="qwen-2.5-32b", groq_api_key=GROQ_API_KEY)


In [63]:

# Initialize Multi-Agent System
agent_executor = initialize_agent(
    tools=[flight_agent, hotel_agent, food_agent, attraction_agent, weather_agent, budget_agent],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=ConversationBufferMemory(memory_key="chat_history")
)

# Function to Generate a Travel Plan
def plan_trip(destination, days=3):
    query = f"Plan a {days}-day trip to {destination} with details on flights, hotels, food, attractions, weather, and budget."
    response = agent_executor.run(query)
    return response


In [64]:

# Example Usage
destination = "Paris"
days = 5
trip_plan = plan_trip(destination, days)
print(trip_plan)




> Entering new AgentExecutor chain...
Thought: To plan a 5-day trip to Paris, I need to gather information on flights, hotels, food, attractions, weather, and budget. I will start with flights.
Action: Flight Search Agent
Action Input: Paris, 5 days from today
Observation: No results found.
Thought:Thought: It's possible that there are no direct flights or availability for the specified dates. I should try a different date range or consider connecting flights.
Action: Flight Search Agent
Action Input: Paris, 5 days from next week, including connecting flights
Observation: No results found.
Thought:Thought: It seems there might be an issue with the specific dates or availability. I'll broaden the search to a more flexible date range to ensure I can find a flight.
Action: Flight Search Agent
Action Input: Paris, flexible dates within the next month, including connecting flights
Observation: No results found.
Thought:Thought: It appears there might be an issue with the flight search age


### **📝 How This Works**
1. **Each agent searches the web** using Serper API for:
   - Flights ✈️  
   - Hotels 🏨  
   - Food 🍽️  
   - Attractions 🗺️  
   - Weather 🌤️  
   - Budget 💰  

2. **The results are fed to Gemini LLM**, which organizes them into a structured itinerary.

3. **Outputs a formatted day-by-day plan** (if the trip spans multiple days).

---
